# Exercise 1- hmmlearn Tutorial

Hidden Markov Model (HMMs) is a generative probabilistic model, in which a sequence of observable `X` variables is generated by a sequence of internal hidden states `Z`. **The hidden states are not observed directly**. The transitions between hidden states are assumed to have the form of a (first-order) _Markov chain_. They can be specified by the **start probability vector π** and a **transition probability matrix A**. The **emission probability of an observable** can be any distribution with **parameters θ conditioned on the current hidden state**. The HMM is completely determined by: `π`, `A` and `θ`



**There are three fundamental problems for HMMs:**

* Given the model parameters and observed data, estimate the optimal sequence of hidden states -  solved by **Viterbi algorithm**
* Given the model parameters and observed data, calculate the model likelihood - solved by **Forward-Backward algorithm**
* Given just the observed data, estimate the model parameters - solved by an **iterative Expectation-Maximization (EM) algorithm, known as the Baum-Welch algorithm**


Note, since the EM algorithm is a gradient-based optimization method, it will generally get stuck in local optima. You should in general try to run fit with various initializations and select the highest scored model.

In [ ]:
import numpy as np
from hmmlearn import hmm
#np.random.seed(42)

## 1.1 - Building HMM by passing the parameters

You can build a HMM instance by passing the parameters described above to the constructor. Then, you can **generate samples from the HMM by calling sample()**.

_Note: The transition matrix does not need to be ergodic, which mean that not all states need to be be visited._

In [2]:
model = hmm.GaussianHMM(n_components=3, covariance_type="full") # HMM with Gaussian Emissions

model.startprob_ = np.array([0.6, 0.3, 0.1])

model.transmat_ = np.array([[0.7, 0.2, 0.1],
                            [0.3, 0.5, 0.2],
                            [0.3, 0.3, 0.4]])

model.means_ = np.array([[0.0, 0.0], [3.0, -3.0], [5.0, 10.0]])

model.covars_ = np.tile(np.identity(2), (3, 1, 1))

In [3]:
X, Z = model.sample(100)

print('Observables:\n', X)
print('Hidden states:\n', Z)

Observables:
 [[-8.99038305e-02 -5.51807853e-01]
 [-1.51572172e+00  5.44575800e-01]
 [ 5.05052855e+00  1.06593581e+01]
 [-3.60839833e-01  6.90650853e-01]
 [-7.45096640e-01  4.41927720e-01]
 [ 3.33652507e+00 -1.30582979e+00]
 [-1.40452059e+00 -1.00709642e+00]
 [-9.96964389e-01 -1.54331437e+00]
 [-4.77319439e-01  9.84565935e-01]
 [-4.18570782e-01 -7.36044918e-01]
 [ 3.51686612e+00 -1.95173229e+00]
 [ 3.47073750e+00 -6.10557100e-01]
 [ 6.07692758e+00  9.09115541e+00]
 [ 2.20136968e+00 -2.29346759e+00]
 [ 5.54528342e+00  1.01391621e+01]
 [ 2.11267234e-01  5.94261107e-01]
 [ 3.40597036e-01 -1.24639220e-01]
 [ 9.08491181e-03 -1.73313183e+00]
 [-2.34923666e-01  1.58722952e-01]
 [ 1.41329253e+00 -2.87352859e+00]
 [-1.78355798e-01 -2.48913870e+00]
 [ 3.03644833e+00  9.26225420e+00]
 [ 2.71105898e+00 -2.40386686e+00]
 [ 2.38362467e+00 -1.88454770e+00]
 [-1.26180487e+00  9.83502087e-01]
 [ 6.56961509e-01 -3.55689240e-01]
 [ 4.41171633e+00  1.04722106e+01]
 [ 2.06074656e+00  2.40249766e-01]
 [ 4.9

## 1.2- Building HMM by fiting the with observables

Lets now fit the model given a sequence of the observablse X (_from last example_) and get the optimal hidden states:

In [4]:
remodel = hmm.GaussianHMM(n_components=3, covariance_type="full", n_iter=100)
remodel.fit(X)

# The inferred optimal hidden states can be obtained by calling predict() method. 
Z2 = remodel.predict(X)

In [5]:
Z2

array([1, 1, 2, 1, 1, 0, 1, 1, 1, 1, 0, 0, 2, 0, 2, 1, 1, 1, 1, 1, 1, 2,
       0, 0, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 1, 0, 0, 0, 1, 1, 0, 2, 0, 1, 0, 2, 2, 2, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 2, 1, 1, 0, 0, 0, 0, 0, 2, 2, 2,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [6]:
# Since the model may not converge in the given n_iter, lets use the monitor_ to control the convergence
remodel.monitor_

ConvergenceMonitor(
    history=[-503.86364285381774, -403.5260741249493, -366.05925059941706, -356.35668212163796, -354.94196644739765, -354.1849616833512, -353.8969364557909, -353.79135983942587, -353.7503393038042, -353.73427598627563, -353.7280213188997],
    iter=11,
    n_iter=100,
    tol=0.01,
    verbose=False,
)

In [7]:
remodel.monitor_.converged

True

The model converged within the 100 iterations. The score of the model can be calculated by the **score()** method.

In [8]:
remodel.score(X)

-353.7256001822194

## 1.3- HMM fitted with multiple sequences

Lets now fit the model with two sequence of observables and get the optimal hidden states

In [9]:
X1 = [[0.5], [1.0], [-1.0], [0.42], [0.24]]
X2 = [[2.4], [4.2], [0.5], [-0.24]]

In [10]:
X = np.concatenate([X1, X2])
lengths = [len(X1), len(X2)]

In [11]:
multihmm = hmm.GaussianHMM(n_components=3)

multihmm.fit(X, lengths)

Fitting a model with 14 free scalar parameters with only 9 data points will result in a degenerate solution.


GaussianHMM(n_components=3)

In [12]:
print('Optimal states for X1:\n', multihmm.predict(X1))
print('Optimal states for X2:\n', multihmm.predict(X2))

Optimal states for X1:
 [0 0 0 0 0]
Optimal states for X2:
 [1 2 0 0]


In [13]:
print('Score for X1:\n', multihmm.score(X1))
print('Score for X2:\n', multihmm.score(X2))

Score for X1:
 -5.8173355766777055
Score for X2:
 0.8644483012618374


In [14]:
multihmm.monitor_.converged

True


# Exercise 2 - HMM in a simple problem

Lets use hmmlearn to model the occasionally dishonest casino problem. This problem needs to be silved using the hmm.MultinomialHMM() 

a) construct the model assuming all parameters known.

b) train the model assuming the transition probability matrix is unknown.

c) train the model assuming the emission probability matrix is unknown.

d) train the model assuming both transition and emission matrices are unkonwn.

e) train the model assuming all the parameters unknown, including the number of states (suggestion: try with the number of states from 1 to 4 and compare the results of the score function).

<img src="casino.png">

## a) construct the model assuming all parameters known.

From the image above it is possible to get all the parameters for the 2 possible states (normal die and loaded die).

In [ ]:
# Matrix of transition probabilities between states from the green arrows above
transmat = np.array([[0.95, 0.05], [0.1, 0.9]])

# Probability of emitting a given symbol when in each state - a number in the die
emitmat_1 = np.array([1/6, 1/6, 1/6, 1/6, 1/6, 1/6]) # first die - fair
emitmat_2 = np.array([1/10, 1/10, 1/10, 1/10, 1/10, 5/10]) # second die - loaded

emitmat = np.array([emitmat_1, emitmat_2])


# Initial state occupation distribution for each die from the Dice observations
N_observations = 60
startprob_1 = 24/N_observations # fair die
startprob_2 = 36/N_observations # loaded die

startprob = np.array([startprob_1, startprob_2])


Since all the parameters are known and the model constructed, it is possible to create a sample of obervations and hidden states using the sample() method. Lets construct the model and get a random sample using the `sample()` method:

In [6]:
# the model is constructed with 2 components - the two dice
hmm_a =  hmm.MultinomialHMM(n_components=2)

# Matrix of transition probabilities between states.
hmm_a.transmat_ = transmat

# Initial state occupation distribution.
hmm_a.startprob_ = startprob

#Probability of emitting a given symbol when in each state.
hmm_a.emissionprob_ = emitmat

In [7]:
# getting a random sample of 60
X_a, Z_a = hmm_a.sample(60, random_state = 0)

In [8]:
import pandas as pd
results = pd.DataFrame(np.concatenate(([X_a.T[0]], [Z_a]))).transpose()
results[1] = results[1].map({0: 'F', 1: 'L'})
results.rename({0:'Rolls', 1:'Dice (a)'}, axis=1, inplace=True)
results['Rolls'] = results['Rolls'] +1 
print('Results for sampled Rolls and Dice (hidden states)')

#printing just the first 50 sampled
print(results[:50].to_markdown())

Results for sampled Rolls and Dice (hidden states)
|    |   Rolls | Dice (a)   |
|---:|--------:|:-----------|
|  0 |       6 | L          |
|  1 |       6 | L          |
|  2 |       6 | L          |
|  3 |       6 | L          |
|  4 |       4 | L          |
|  5 |       6 | L          |
|  6 |       6 | L          |
|  7 |       1 | F          |
|  8 |       5 | F          |
|  9 |       6 | F          |
| 10 |       6 | L          |
| 11 |       6 | L          |
| 12 |       6 | L          |
| 13 |       6 | L          |
| 14 |       5 | L          |
| 15 |       6 | L          |
| 16 |       6 | L          |
| 17 |       4 | F          |
| 18 |       4 | F          |
| 19 |       5 | F          |
| 20 |       3 | F          |
| 21 |       1 | F          |
| 22 |       5 | F          |
| 23 |       1 | F          |
| 24 |       3 | F          |
| 25 |       3 | F          |
| 26 |       2 | L          |
| 27 |       2 | L          |
| 28 |       3 | L          |
| 29 |       3 | L 

Hmmlearn has two distict ways to score the model:

*  `decode` method gives you the most likely sequence of **hidden states for an observation sequence**, as well as **its log probability**, using (by default) the **Viterbi algorithm**. 
*  `score` method gives you the probability of the observed state sequence, marginalizing out all the possible hidden states. (_from documentation_)

In [19]:
import math
score = hmm_a.score(X_a)
logods, Z_a2 =  hmm_a.decode(X_a)
Z_a2_=pd.DataFrame(Z_a2)[0].map({0: 'F', 1: 'L'}).transpose()

print(f'The probability of the observed Rolls is {math.exp(score)*100}%\n\n')

print(f'The most likely hidden states (0:F | 1:L) is:\n {Z_a2}\n')
print(f'The probability of those hidden states is {math.exp(logods)*100}%')

The probability of the observed Rolls is 5.6519946513321525e-42%


The most likely hidden states (0:F | 1:L) is:
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

The probability of those hidden states is 5.451664403057156e-43%


We observed that the most likely hidden states given dy `decode()` are always the same for the same observables, which makes sense. The `predict()` method will be used to evaluate how close of the most likely hidden states the model is when predicting hidden states from a sample obervation.

The sample observation used beyond this point will be `X_a`, being its most likely hidden states `Z_a2`

## b) train the model assuming the transition probability matrix is unknown.

If any of the paremeters is unkknown, the model needs to be fitted in order to get a prediction of the unknown parameter(s):

In [13]:
from numpy import random
import numpy as np
from sklearn.metrics import accuracy_score

The original transition matrix that we're trying to fit is:

In [21]:
print(hmm_a.transmat_ )

[[0.95 0.05]
 [0.1  0.9 ]]


Even though there were no problems in converging it was decided to test initializing the transition matrix with different conditions, as documented in hmmlearn docs

In [22]:
def fit_trans(X_a,Z_a,Z_a2,n_runs = 5):  
    for i in range(n_runs):
        hmm_b = hmm.MultinomialHMM(n_components=2, init_params = "", params = "t",n_iter=1000, random_state=0)

        #random initialization of transition matrix. Each run will be different
        hmm_b.transmat_ = np.random.dirichlet(np.ones(2),size=2)


        # Initial state occupation distribution.
        hmm_b.startprob_ = startprob

        #Probability of emitting a given symbol when in each state.
        hmm_b.emissionprob_ = emitmat

        if i == 0 :
            score = hmm_b.score(X_a)

        #lets fit the model from the observations of a)
        hmm_b.fit(X_a)

        #recording the predictions and results from the best score
        score_run = hmm_b.score(X_a)
        if score <= score_run  :
            score = score_run
            transmat_result = hmm_b.transmat_
            result = hmm_b.predict(X_a)
    print("The fitted transition matrix is:\n",transmat_result)
    print("\n \nThis fit, predicts the sampled hidden states from a) with an accuracy of:",accuracy_score(Z_a,result))
    print("\n \nThis fit, predicts the most likely hidden states with an accuracy of:", accuracy_score(Z_a2,result))

In [23]:
fit_trans(X_a,Z_a,Z_a2,10)

The fitted transition matrix is:
 [[9.99946822e-01 5.31780818e-05]
 [5.77040774e-02 9.42295923e-01]]

 
This fit, predicts the sampled hidden states from a) with an accuracy of: 0.7

 
This fit, predicts the most likely hidden states with an accuracy of: 1.0


* The predicted matrix is close to the original transition matrix.
* Increasing the number of runs was tested, but it did not improve the results since the problem already converges well. Perhaps with a longer input sequence a longer number of runs they will improve.
* accuracy was used as a means to verify how close the prediction of the hidden states calculated using the fitted transition matrix were to a)the hidden states sampled from the model and b) the most likely hidden states.
* The prediction had 100% accuracy predicting the most likely hidden states but was only 70% accurate when predicting the sampled hidden states from a)


Let's verify what happens when the sample is larger:

In [24]:
X_b, Z_b = hmm_a.sample(2000, random_state = 0)
log_b, Z_b2 = hmm_a.decode(X_b)

In [25]:
fit_trans(X_b,Z_b, Z_b2,40)

The fitted transition matrix is:
 [[0.90269027 0.09730973]
 [0.15938903 0.84061097]]

 
This fit, predicts the sampled hidden states from a) with an accuracy of: 0.7335

 
This fit, predicts the most likely hidden states with an accuracy of: 0.9275


**Take-Aways:**

* With a larger sample, the model fitted the transition matrix slightly worse, which is somewhat unexpected since it has more data to learn from and both accuracies went down, which makes sense since the larger sequence, the lower is the probability of it happening.

## c) train the model assuming the emission probability matrix is unknown.

In [26]:
def fit_emiss(X_a,Z_a,Z_a2,n_runs = 5):  
    for i in range(n_runs):
        hmm_b = hmm.MultinomialHMM(n_components=2, init_params = "", params = "e",n_iter=10000, random_state=0)


        #initialization of transition matrix
        hmm_b.transmat_ = transmat


        # Initial state occupation distribution.
        hmm_b.startprob_ = startprob

        #Probability of emitting a given symbol when in each state.
        hmm_b.emissionprob_ = np.random.dirichlet(np.ones(6),size=2)

        if i == 0 :
            score = hmm_b.score(X_a)

        #lets fit the model from the observations of a)
        hmm_b.fit(X_a)

        #recording the predictions and results from the best score
        if score <= hmm_b.score(X_a)  :
            score = hmm_b.score(X_a)
            emission_result = hmm_b.emissionprob_
            result = hmm_b.predict(X_a)


    print("The fitted emission matrix is:\n",emission_result)
    print("\n \nThis fit, predicts the sampled hidden states from a) with an accuracy of:",accuracy_score(Z_a,result))
    print("\n \nThis fit, predicts the most likely hidden states with an accuracy of:", accuracy_score(Z_a2,result))

In [27]:
fit_emiss(X_a,Z_a, Z_a2,n_runs = 10)

The fitted emission matrix is:
 [[2.22830531e-01 2.76330161e-01 1.65800949e-01 6.41276277e-02
  2.12759108e-01 5.81516226e-02]
 [3.93169074e-02 7.23942762e-06 7.78426689e-09 1.96512072e-01
  9.66183801e-02 6.67545393e-01]]

 
This fit, predicts the sampled hidden states from a) with an accuracy of: 0.5833333333333334

 
This fit, predicts the most likely hidden states with an accuracy of: 0.8833333333333333


**Lets see what happens when we increase the sampled data in this case:**

In [218]:
X_b, Z_b = hmm_a.sample(2000, random_state = 0)
log_b, Z_b2 = hmm_a.decode(X_b)

In [219]:
fit_emiss(X_b,Z_b,Z_b2,n_runs = 10)

The fitted emission matrix is:
 [[0.14740446 0.18250059 0.17803454 0.14143779 0.15472064 0.19590198]
 [0.11237734 0.08687241 0.08377763 0.13573614 0.09669558 0.48454091]]

 
This fit, predicts the sampled hidden states from a) with an accuracy of: 0.6965

 
This fit, predicts the most likely hidden states with an accuracy of: 0.9375


**Take-Aways:**

* Fitting 10 variables (not 12 because they all sum to one) with only 60 data points would always be a challenge and it shows in the results. This explains why increasing the length had the opossite effect as in case of item b)
* If the length of the sequence is increased then the model has no problems in finding a better estimate, hence the better accuracies

## d) train the model assuming both transition and emission matrices are unkonwn.

In [47]:
def fit_emiss_trans(X_a,Z_a,Z_a2,n_runs = 10):  
    for i in range(n_runs):
        hmm_b = hmm.MultinomialHMM(n_components=2, init_params = "", params = "e",n_iter=50000, random_state=0)

        #initialization of transition matrix
        hmm_b.transmat_ = np.random.dirichlet(np.ones(2),size=2)

        # Initial state occupation distribution.
        hmm_b.startprob_ = startprob

        #Probability of emitting a given symbol when in each state.
        hmm_b.emissionprob_ = np.random.dirichlet(np.ones(6),size=2)

        if i == 0 :
            score = hmm_b.score(X_a)

        #lets fit the model from the observations of a)
        hmm_b.fit(X_a)

        #recording the predictions and results from the best score
        if score <= hmm_b.score(X_a)  :
            score = hmm_b.score(X_a)
            emission_result = hmm_b.emissionprob_
            transmat_result = hmm_b.transmat_
            result = hmm_b.predict(X_a)


    print("The fitted emission matrix is:\n",emission_result)
    print("\nThe transition emission matrix is:\n",transmat_result)
    print("\n \nThis fit, predicts the sampled hidden states from a) with an accuracy of:",accuracy_score(Z_a,result))
    print("\n \nThis fit, predicts the most likely hidden states with an accuracy of:", accuracy_score(Z_a2,result))

In [50]:
fit_emiss_trans(X_a,Z_a,Z_a2,n_runs = 200)

The fitted emission matrix is:
 [[2.25521175e-01 2.78676023e-01 1.67218033e-01 6.16692077e-02
  2.12731063e-01 5.41844980e-02]
 [3.76474491e-02 3.07936527e-05 1.52027528e-10 1.98486167e-01
  9.81368479e-02 6.65698742e-01]]

The transition emission matrix is:
 [[0.92907569 0.07092431]
 [0.10813215 0.89186785]]

 
This fit, predicts the sampled hidden states from a) with an accuracy of: 0.5833333333333334

 
This fit, predicts the most likely hidden states with an accuracy of: 0.8833333333333333


To have results this good, the number of runs has to be increased. Still, it is remarkable how good they are considering the short sequence.

**Lets see how it performs with a bigger sample:**

In [11]:
X_b, Z_b = hmm_a.sample(2000, random_state = 0)
log_b, Z_b2 = hmm_a.decode(X_b)

In [54]:
fit_emiss_trans(X_b,Z_b,Z_b2,n_runs = 200)

The fitted emission matrix is:
 [[0.14851327 0.19115028 0.18638599 0.13903718 0.15842695 0.17648633]
 [0.11458793 0.08387231 0.0811005  0.14024374 0.09735335 0.48284218]]

The transition emission matrix is:
 [[0.92078341 0.07921659]
 [0.12927059 0.87072941]]

 
This fit, predicts the sampled hidden states from a) with an accuracy of: 0.736

 
This fit, predicts the most likely hidden states with an accuracy of: 0.963


**Take-Aways:**

* The results with both emission and transmition matrices being fitted are more dependent of initial conditions and to achieve reasonably consistent results, it is necessary to increase the number of runs quite a bit. Otherwise every set of runs would yield results with accuracies that varied from 10% to 90%, which is a lot.

## e) train the model assuming all the parameters unknown, including the number of states (suggestion: try with the number of states from 1 to 4 and compare the results of the score function).

In [40]:
def fit_emiss_trans_states(X_a,Z_a,Z_a2,n_runs = 10):  
    scores = []
    transitions = []
    emissions = []
    start_probs = []
    for n_state in range(4):
        for i in range(n_runs):
            hmm_b = hmm.MultinomialHMM(n_components=n_state+1, init_params = "", params = "ets",n_iter=20000, random_state=0)

            #initialization of transition matrix
            hmm_b.transmat_ = np.random.dirichlet(np.ones(n_state+1),size=n_state+1)

            # Initial state occupation distribution.
            hmm_b.startprob_ = np.random.dirichlet(np.ones(n_state+1))

            #Probability of emitting a given symbol when in each state.
            hmm_b.emissionprob_ = np.random.dirichlet(np.ones(6),size=n_state+1)

            if i == 0 :
                score_record = hmm_b.score(X_a)

            #lets fit the model from the observations of a)
            hmm_b.fit(X_a)
            
            #recording the predictions and results from the best score
            score = hmm_b.score(X_a)
            
            if score_record <= score:
                score_record = score
                emission_result = hmm_b.emissionprob_
                transmat_result = hmm_b.transmat_
                prob_result = hmm_b.startprob_
                result = hmm_b.predict(X_a)  
        scores.append(score_record)
        emissions.append(emission_result)
        transitions.append(transmat_result)
        start_probs.append(prob_result)

            
        
    
    print('The scores were:')
    display(pd.DataFrame({'N_states':[1,2,3,4],'Max_score':scores}) )
    
    
    print('The best parameters were: \n\n')
    
    print('Number of states:')
    n_state_idx = np.argmax(scores)
    print(n_state_idx+1)
    
    print("\nThe fitted emission matrix is:\n",emissions[n_state_idx] )
    print("\nThe transition emission matrix is:\n",transitions[n_state_idx])
    print("\nThe starting probability matrix is:\n",start_probs[n_state_idx])

    print("\n \nThis fit, predicts the sampled hidden states from a) with an accuracy of:",accuracy_score(Z_a,result))
    print("\n \nThis fit, predicts the most likely hidden states with an accuracy of:", accuracy_score(Z_a2,result))
    

In [44]:
fit_emiss_trans_states(X_a,Z_a,Z_a2,n_runs = 100)

The scores were:


,N_states,Max_score
0,1,-103.435331
1,2,-91.929356
2,3,-86.256880
3,4,-82.226421


The best parameters were: 


Number of states:
4

The fitted emission matrix is:
 [[3.29128719e-01 2.06240400e-01 4.64932070e-02 2.78472337e-10
  3.29234425e-01 8.89032496e-02]
 [1.91036437e-10 5.05919880e-01 4.94080120e-01 8.98199160e-14
  1.15012032e-31 3.62016305e-17]
 [4.48237590e-02 2.49545714e-11 3.73958173e-28 1.55038026e-01
  8.94967309e-02 7.10641484e-01]
 [4.34969077e-05 3.14229164e-08 2.58802078e-08 9.99418120e-01
  5.38314868e-04 1.10169325e-08]]

The transition emission matrix is:
 [[9.12989748e-01 4.31365822e-02 4.38736702e-02 4.09317108e-12]
 [7.04478232e-10 7.97055221e-01 1.72133926e-11 2.02944778e-01]
 [1.75316590e-05 4.46328394e-02 9.09245622e-01 4.61040074e-02]
 [8.53671555e-01 3.18257324e-33 4.38987024e-05 1.46284547e-01]]

The starting probability matrix is:
 [7.42353021e-046 1.46368654e-225 1.00000000e+000 9.07284342e-067]

 
This fit, predicts the sampled hidden states from a) with an accuracy of: 0.36666666666666664

 
This fit, predicts the most likely hidden s

The error seems to decrease monotonically with the number of states. Intuitively, when we increase the number of states, we are also increasing the number of parameters to be trained and so the model has more freedom to adapt to the data. It's still somewhat unexpected. Let's try with a larger sequence

In [45]:
X_b, Z_b = hmm_a.sample(2000, random_state = 0)
log_b, Z_b2 = hmm_a.decode(X_b)

In [46]:
fit_emiss_trans_states(X_b,Z_b,Z_b2,n_runs = 20)

The scores were:


,N_states,Max_score
0,1,-3482.686405
1,2,-3463.097766
2,3,-3458.752215
3,4,-3447.997810


The best parameters were: 


Number of states:
4

The fitted emission matrix is:
 [[1.40738767e-01 1.92618941e-01 1.94714335e-01 1.39108739e-01
  1.65778848e-01 1.67040370e-01]
 [2.24009908e-06 7.06373971e-02 1.48928514e-02 1.95930109e-01
  7.29349447e-02 6.45602457e-01]
 [3.02404598e-01 7.18114293e-02 1.33166026e-01 1.82060521e-01
  1.23633041e-03 3.09321095e-01]
 [2.51452983e-06 1.33090502e-01 5.70527449e-02 3.67977393e-02
  2.31039038e-01 5.42017461e-01]]

The transition emission matrix is:
 [[8.88726022e-01 5.39672955e-04 1.07948968e-01 2.78533643e-03]
 [6.75667368e-02 4.39984497e-04 7.85471137e-01 1.46522142e-01]
 [3.09753937e-01 4.60945411e-02 5.55649600e-03 6.38595025e-01]
 [2.11501460e-02 7.68099369e-01 2.10411787e-01 3.38698337e-04]]

The starting probability matrix is:
 [0.00000000e+000 5.07317117e-052 7.14359082e-107 1.00000000e+000]

 
This fit, predicts the sampled hidden states from a) with an accuracy of: 0.578

 
This fit, predicts the most likely hidden states with an 

Even with such a long sequence, the same effect was observed. Note that the accuracy was also a lot worse compared with the previous question items